In [ ]:
!pip install numpy==1.17.4
!pip install nltk==3.4.5
!pip install torchtext==0.4.0
!pip install scikit_learn==0.23.2
!pip install spacy==2.3.5
!pip install textblob==0.15.3
!pip install torch==1.6.0 
!pip install torchvision==0.7.0
!pip install tqdm
!pip install underthesea

In [1]:
import nltk
nltk.download('wordnet')

import os
import math
import random
import argparse
from pathlib import Path
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import BucketIterator

from torchtext.data import Field, Example, Dataset
import re
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
import torch.nn.functional as F

from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import time
from torchtext.data import BucketIterator
import gc

path = '/kaggle/working/' #colab: /content/ or bla bla...
import sys
sys.argv=['']
del sys

In [2]:
def parse_args():
    """Add arguments to parser"""
    parser = argparse.ArgumentParser(description='Verbalization dataset baseline models.')
    parser.add_argument('--model', default=RNN_NAME, type=str,
                        choices=[RNN_NAME], help='model to train the dataset')
    parser.add_argument('--input', default=QUESTION, type=str,
                        choices=[QUESTION], help='use question as input')
    parser.add_argument('--attention', default=ATTENTION_2, type=str,
                        choices=[ATTENTION_2], help='attention layer for rnn model')
    args = parser.parse_args()
    return args

def set_SEED():
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

In [3]:
"""Constants for the baseline models"""
SEED = 42
QUESTION = 'question'

RNN_NAME = 'rnn'
CNN_NAME = 'cnn'
TRANSFORMER_NAME = 'transformer'

ATTENTION_1 = 'bahdanau'
ATTENTION_2 = 'luong'

GPU = 'gpu'
CPU = 'cpu'
CUDA = 'cuda'

CHECKPOINT_PATH = '/model/'

ANSWER_TOKEN = '<ans>'
ENTITY_TOKEN = '<ent>'
EOS_TOKEN = '<eos>'
SOS_TOKEN = '<sos>'
PAD_TOKEN = '<pad>'

SRC_NAME = 'src'
TRG_NAME = 'trg'

In [4]:
class Chechpoint(object):
    """Checkpoint class"""
    @staticmethod
    def save(model, path):
        """Save model using name"""
        name = f'{model.name}.pt'
        torch.save(model.state_dict(), path+name)

    @staticmethod
    def load(model,path, name):
        """Load model using name"""
        #name = f'{model.name}.pt'
        model.load_state_dict(torch.load(path+name))
        return model

In [5]:
from underthesea import word_tokenize

class VerbalDataset(object):
    """VerbalDataset class"""
#     TOKENIZE_SEQ = lambda self, x: x.replace("?", " ?").\
#                                      replace(".", " .").\
#                                      replace(",", " ,").\
#                                      replace("'", " '").\
#                                      split()
                                         
    def __init__(self,train,test):
        self.train = train
        self.test = test
        self.train_data = None
        self.valid_data = None
        self.test_data = None
        self.src_field = None
        self.trg_field = None

    def _extract_question_answer2(self, train, test):
        return [[data['question'], [data['verbalized_answer'], data['verbalized_answer_2'],data['verbalized_answer_3'],data['verbalized_answer_4'],data['verbalized_answer_5'],data['verbalized_answer_6'],data['verbalized_answer_7'],data['verbalized_answer_8']]] for data in train], \
                [[data['question'], [data['verbalized_answer'], data['verbalized_answer_2'],data['verbalized_answer_3'],data['verbalized_answer_4'],data['verbalized_answer_5'],data['verbalized_answer_6'],data['verbalized_answer_7'],data['verbalized_answer_8']]] for data in test]

    def _extract_question_answer(self, train, test):
        return [[data['Question'], data['Answer']] for data in train], \
                [[data['Question'], data['Answer']] for data in test]


    def _make_torchtext_dataset(self, data, fields):
        examples = [Example.fromlist(i, fields) for i in data]
        return Dataset(examples, fields)

    def load_data_and_fields(self, ):
        """
        Load verbalization data
        Create source and target fields
        """
        train, test, val = [], [], []
        
        train = self.train
        test = self.test

        # split test data to val-test
        test, val = train_test_split(test, test_size=0.5, shuffle=False)

        # create fields
        self.src_field = Field(tokenize=word_tokenize,
                               init_token=SOS_TOKEN,
                               eos_token=EOS_TOKEN,
                               lower=True,
                               include_lengths=True,
                               batch_first=True)
        
        self.trg_field = Field(tokenize=word_tokenize,
                               init_token=SOS_TOKEN,
                               eos_token=EOS_TOKEN,
                               lower=True,
                               batch_first=True)

        fields_tuple = [(SRC_NAME, self.src_field), (TRG_NAME, self.trg_field)]

        # create toechtext datasets
        self.train_data = self._make_torchtext_dataset(train, fields_tuple)
        self.valid_data = self._make_torchtext_dataset(val, fields_tuple)
        self.test_data = self._make_torchtext_dataset(test, fields_tuple)

        # build vocabularies
        self.src_field.build_vocab(self.train_data, min_freq=2)
        self.trg_field.build_vocab(self.train_data, min_freq=2)
        print("i am field tuple",fields_tuple)

    def get_data(self):
        """Return train, validation and test data objects"""
        return self.train_data, self.valid_data, self.test_data

    def get_fields(self):
        """Return source and target field objects"""
        return self.src_field, self.trg_field

    def get_vocabs(self):
        """Return source and target vocabularies"""
        #print('self, trg field vocab: ', self.trg_field.vocab)
        return self.src_field.vocab, self.trg_field.vocab

In [6]:
set_SEED()
args = parse_args()

df = pd.read_csv('/kaggle/input/covid19-qa/Covid19_QA(4300).csv')
df = df[['Question','Answer']]
train, test = train_test_split(df.values,train_size=0.814,random_state=42)

dataset = VerbalDataset(train,test)

dataset.load_data_and_fields()
src_vocab, trg_vocab = dataset.get_vocabs()
train_data, valid_data, test_data = dataset.get_data()

In [11]:
print('--------------------------------')
print(f"Training data: {len(train_data.examples)}")
print(f"Evaluation data: {len(valid_data.examples)}")
print(f"Testing data: {len(test_data.examples)}")
print('--------------------------------')
print(f'Question example: {train_data.examples[2].src}\n')
print(f'Answer example: {train_data.examples[2].trg}')
print('--------------------------------')
print(f"Unique tokens in questions vocabulary: {len(src_vocab)}")
print(f"Unique tokens in answers vocabulary: {len(trg_vocab)}")
print('--------------------------------')

# Sequence-to-Sequence Model

### Layers

In [16]:
def RNN(cell_name):
    if cell_name.lower() == 'lstm':
        return LSTM
    elif cell_name.lower() == 'gru':
        return GRU
    else:
        raise ValueError(f"Unsupported RNN Cell: {cell_name}")

def Embedding(num_embeddings, embedding_dim, padding_idx):
    """Embedding layer"""
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.uniform_(m.weight, -0.1, 0.1)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m

def Linear(in_features, out_features, bias=True):
    """Linear layer"""
    m = nn.Linear(in_features, out_features, bias=bias)
    m.weight.data.uniform_(-0.1, 0.1)
    if bias:
        m.bias.data.uniform_(-0.1, 0.1)
    return m

def LSTM(input_size, hidden_size, **kwargs):
    """LSTM layer"""
    m = nn.LSTM(input_size, hidden_size, **kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m

def GRU(input_size, hidden_size, **kwargs):
    """GRU layer"""
    m = nn.GRU(input_size, hidden_size, **kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m

### Encoder

In [17]:
class Encoder(nn.Module):
    """Encoder"""
    def __init__(self, vocabulary, device, hidden_size=512, num_layers=2,
                 bidirectional=True, cell_name='gru', dropout=0.5):
        super().__init__()
        input_dim = len(vocabulary)
        self.num_layers = num_layers
        self.pad_id = vocabulary.stoi[PAD_TOKEN]
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.device = device
        self.rnn_cell = RNN(cell_name)

        self.embedding = Embedding(input_dim, self.hidden_size, self.pad_id)
        self.dropout = dropout

        self.rnn = self.rnn_cell(
            input_size=self.hidden_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True,
            bidirectional=self.bidirectional,
            dropout=self.dropout if self.num_layers > 1 else 0.
        )

    def forward(self, src_tokens, **kwargs):
        """
        Forward Encoder
        Args:
            src_tokens (LongTensor): (batch, src_len)
            src_lengths (LongTensor): (batch)
        Returns:
            x (LongTensor): (src_len, batch, hidden_size * num_directions)
            hidden (LongTensor): (batch, enc_hid_dim)
        """
        src_lengths = kwargs.get('src_lengths', '')

        embedded = self.embedding(src_tokens)
        embedded = F.dropout(embedded, p=self.dropout, training=self.training)

        embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_lengths, batch_first=True)
        output, hidden = self.rnn(embedded)

        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        output = F.dropout(output, p=self.dropout, training=self.training)

        if isinstance(hidden, tuple):
            hidden = tuple([self._cat_directions(h) for h in hidden])
        else:
            hidden = self._cat_directions(hidden)

        return output, hidden

    def _cat_directions(self, h):
        """
        If the encoder is bidirectional, do the following transformation.
        (#directions * #layers, #batch, hidden_size) -> (#layers, #batch, #directions * hidden_size)
        """
        if self.bidirectional:
            h = torch.cat([h[0:h.size(0):2], h[1:h.size(0):2]], 2)
        return h


### Attention

In [ ]:
class Attention(nn.Module):
    """Attention"""
    def __init__(self, input_embed, source_embed, output_embed):
        super().__init__()
        self.linear_in = Linear(input_embed, source_embed)
        self.linear_out = Linear(input_embed+source_embed, output_embed)

    def forward(self, output, context, mask):
        """
        Forward Attention
        """
        # input: bsz x input_embed_dim
        # source_hids: srclen x bsz x source_embed_dim

        input = output.squeeze(1)
        source_hids = context.permute(1, 0, 2)

        x = self.linear_in(input)

        # compute attention
        attn_scores = (source_hids * x.unsqueeze(0)).sum(dim=2)

        # don't attend over padding
        attn_scores = attn_scores.float().masked_fill(mask == 0, float('-inf'))

        attn_scores = F.softmax(attn_scores, dim=0)  # srclen x bsz

        # sum weighted sources
        x = (attn_scores.unsqueeze(2) * source_hids).sum(dim=0)

        x = torch.cat((x, input), dim=1)
        x = self.linear_out(x)
        x = torch.tanh(x)

        return x, attn_scores


### Decoder

In [ ]:
class Decoder(nn.Module):
    """Decoder"""
    def __init__(self, vocabulary, device, hidden_size=512, num_layers=2,
                 max_len=50, cell_name='gru', dropout=0.5):
        super().__init__()
        self.output_dim = len(vocabulary)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.max_length = max_len
        self.device = device
        self.eos_id = vocabulary.stoi[EOS_TOKEN]
        self.sos_id = vocabulary.stoi[SOS_TOKEN]
        self.pad_id = vocabulary.stoi[PAD_TOKEN]
        self.rnn_cell = RNN(cell_name)

        self.encoder_proj = Linear(hidden_size*2, hidden_size)

        self.embedding = Embedding(self.output_dim, self.hidden_size, self.pad_id)
        self.dropout = dropout

        self.rnn = self.rnn_cell(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=self.num_layers,
            batch_first=True,
            dropout=self.dropout if num_layers > 1 else 0.
        )

        self.attention = Attention(self.hidden_size, self.hidden_size*2, self.hidden_size)
        self.linear_out = Linear(self.hidden_size, self.output_dim)

    def _decoder_step(self, input_var, hidden, encoder_outputs, mask):
        input_var = input_var.unsqueeze(1)

        embedded = self.embedding(input_var)
        embedded = F.dropout(embedded, p=self.dropout, training=self.training)

        output, hidden = self.rnn(embedded, hidden)
        output = F.dropout(output, p=self.dropout, training=self.training)

        output, attn = self.attention(output, encoder_outputs, mask)
        output = F.dropout(output, p=self.dropout, training=self.training)

        output = self.linear_out(output)
        # output = F.dropout(output, p=self.dropout, training=self.training)
        output = F.log_softmax(output, dim=1)

        return output, hidden, attn

    def forward(self, trg_tokens, encoder_out, **kwargs):
        """
        Forward Decoder
        """
        encoder_out, hidden = encoder_out
        src_tokens = kwargs.get('src_tokens', '')
        teacher_forcing_ratio = kwargs.get('teacher_forcing_ratio', '')
        batch_size, src_length = src_tokens.size()

        if trg_tokens is None:
            teacher_forcing_ratio = 0.
            inference = True
            trg_tokens = torch.zeros((batch_size, self.max_length)).long().\
                                                                      fill_(self.sos_id).\
                                                                      to(self.device)
        else:
            inference = False

        max_length = trg_tokens.shape[1]

        outputs = torch.zeros(max_length, batch_size, self.output_dim).to(self.device)
        attentions = torch.zeros(max_length, batch_size, src_length).to(self.device)

        mask = (src_tokens != self.pad_id).t()

        # check whether encoder has lstm or gru hidden state and
        # project their output to decoder hidden state
        if isinstance(hidden, tuple):
            hidden = [self.encoder_proj(h) for h in hidden] # new_line
        else:
            hidden = self.encoder_proj(hidden)

        # use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        decoder_input = trg_tokens[:, 0]

        # Here we miss the output for position 0
        for i in range(1, max_length):
            output, hidden, attention = self._decoder_step(decoder_input, hidden, encoder_out, mask)
            outputs[i] = output
            attentions[i] = attention.t()
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            decoder_input = trg_tokens[:, i] if use_teacher_forcing else output.argmax(1)

            if inference and decoder_input.item() == self.eos_id and i > 0:
                return outputs[:i] # , attentions[:i]

        return outputs # , attentions

### Seq2Seq

In [18]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, name):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.name = name

    def forward(self, src_tokens, src_lengths, trg_tokens, teacher_forcing_ratio=0.5):
        encoder_out = self.encoder(src_tokens, 
                                   src_lengths=src_lengths)
        
        decoder_out = self.decoder(trg_tokens, encoder_out,
                                   src_tokens=src_tokens,
                                   teacher_forcing_ratio=teacher_forcing_ratio)
        return decoder_out

In [15]:
DEVICE = torch.device(CUDA if torch.cuda.is_available() else CPU)
encoder = Encoder(src_vocab, DEVICE)
decoder = Decoder(trg_vocab, DEVICE)
model = Seq2Seq(encoder, decoder, args.model).to(DEVICE)

parameters_num = sum(p.numel() for p in model.parameters() if p.requires_grad)

print('--------------------------------')
print(f'Model: {args.model}')
print(f'Model input: {args.input}')
if args.model == RNN_NAME:
    print(f'Attention: {args.attention}')
print(f'The model has {parameters_num:,} trainable parameters')
print('--------------------------------')

# Model Training

### Evaluator

In [19]:
class Evaluator(object):
    """Evaluator class"""
    def __init__(self, criterion):
        self.criterion = criterion

    def evaluate(self, model, iterator, teacher_ratio=1.0):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for _, batch in enumerate(iterator):
                src, src_len = batch.src
                trg = batch.trg
                input_trg = trg if model.name == RNN_NAME else trg[:, :-1]
                output = model(src, src_len, input_trg, teacher_ratio)
                trg = trg.t() if model.name == RNN_NAME else trg[:, 1:]
                output = output.contiguous().view(-1, output.shape[-1])
                trg = trg.contiguous().view(-1)
                # output: (batch_size * trg_len) x output_dim
                # trg: (batch_size * trg_len)
                loss = self.criterion(output, trg)
                epoch_loss += loss.item()
        return epoch_loss / len(iterator)

### Trainer

In [20]:
class Trainer(object):
    """Trainer Class"""
    def __init__(self, optimizer, criterion, batch_size, device):
        self.optimizer = optimizer
        self.criterion = criterion
        self.batch_size = batch_size
        self.device = device
        self.evaluator = Evaluator(criterion=self.criterion)

    def _train_batch(self, model, iterator, teacher_ratio, clip):
        model.train()
        epoch_loss = 0
        for _, batch in enumerate(tqdm_notebook(iterator)):
            src, src_len = batch.src
            trg = batch.trg
            self.optimizer.zero_grad()
            input_trg = trg if model.name == RNN_NAME else trg[:, :-1]
            output = model(src, src_len, input_trg, teacher_ratio)
            trg = trg.t() if model.name == RNN_NAME else trg[:, 1:]
            output = output.contiguous().view(-1, output.shape[-1])
            trg = trg.contiguous().view(-1)
            # output: (batch_size * trg_len) x output_dim
            # trg: (batch_size * trg_len)
            torch.cuda.empty_cache()
            loss = self.criterion(output, trg)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            self.optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / len(iterator)

    def _get_iterators(self, train_data, valid_data, model_name):
        return BucketIterator.splits((train_data, valid_data),
                                     batch_size=self.batch_size,
                                     sort_within_batch=True if model_name == RNN_NAME else \
                                                       False,
                                     sort_key=lambda x: len(x.src),
                                     device=self.device)

    def _epoch_time(self, start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs

    def _log_epoch(self, train_loss, valid_loss, epoch, start_time, end_time):
        minutes, seconds = self._epoch_time(start_time, end_time)
        print(f'Epoch: {epoch+1:02} | Time: {minutes}m {seconds}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {np.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {np.exp(valid_loss):7.3f}')

    def _train_epoches(self, model, train_data, valid_data,  path_, num_of_epochs, teacher_ratio, clip):
        best_valid_loss = float('inf')
        # pylint: disable=unbalanced-tuple-unpacking
        train_iterator, valid_iterator = self._get_iterators(train_data, valid_data, model.name)
        for epoch in range(num_of_epochs):
            start_time = time.time()
            train_loss = self._train_batch(model, train_iterator, teacher_ratio, clip)
            valid_loss = self.evaluator.evaluate(model, valid_iterator, teacher_ratio)
            end_time = time.time()
            self._log_epoch(train_loss, valid_loss, epoch, start_time, end_time)
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                Chechpoint.save(model,path_)

    def train(self, model, train_data, valid_data, path_, num_of_epochs=20, teacher_ratio=1.0, clip=1):
        """Train model"""
        self._train_epoches(model, train_data, valid_data, path_, num_of_epochs, teacher_ratio, clip)

### Train model

In [21]:
# create optimizer
optimizer = optim.Adam(model.parameters(),lr=0.001)
# define criterion
criterion = nn.CrossEntropyLoss(ignore_index=trg_vocab.stoi[PAD_TOKEN])
# batch_size
batch_size = 12

trainer = Trainer(optimizer, criterion, batch_size, DEVICE)
trainer.train(model, train_data, valid_data, path, num_of_epochs=1)

# Predict and Evaluation

### BleuScorer (gồm BLEU và METEOR)

In [22]:
import nltk

class BleuScorer(object):
    """Blue scorer class"""
    def __init__(self):
        self.results = []
        self.results_meteor = []
        self.score = 0
        self.meteor_score = 0
        self.instances = 0
        self.meteor_instances = 0

    def example_score(self, reference, hypothesis):
        """Calculate blue score for one example"""
        return nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,weights=(1,0,0,0)) #unigram
    
    def example_score_meteor(self, reference, hypothesis):
        """Calculate blue score for one example"""
        return nltk.translate.meteor_score.single_meteor_score(reference,hypothesis)

    def data_score(self, data, predictor, path):
        """Score complete list of data"""
        results_prelim = []
        for example in data:
            #i = 1
            src = [t.lower() for t in example.src]
            reference = [t.lower() for t in example.trg]
            # loop through example.src and calculate all hypothesis(max. 8) 
            #and calculate blue score average of all hypothesis
            hypothesis = predictor.predict(example.src)
            blue_score = self.example_score(reference, hypothesis)
            meteor_score = self.example_score_meteor(' '.join(reference), ' '.join(hypothesis))
            #print('Blue Score: ',blue_score)
            results_prelim.append({
                'question': '"' + str(src) + '"',
                'reference': reference,
                'hypothesis': hypothesis,
                'blue_score': blue_score,
                'meteor_score': meteor_score
            })
        #print('List length before aggregation',len(results_prelim))

        results = [max((v for v in results_prelim if v['question'] == x), key=lambda y:y['blue_score']) for x in set(v['question'] for v in results_prelim)] 

        with open(path+'result_output.txt', 'w') as f:
            for elem in results:
                f.write("%s\n" % elem)
                self.results.append(elem)
                self.score += elem['blue_score']
                self.meteor_score += elem['meteor_score']
                self.instances += 1
        return self.score / self.instances, self.meteor_score / self.instances

    def average_score(self):
        """Return bleu average score"""
        return self.score / self.instances
    
    def data_meteor_score(self, data, predictor, path):
        """Score complete list of data"""
        results_prelim = []
        for example in data:
            src = [t.lower() for t in example.src]
            reference = [t.lower() for t in example.trg]
            hypothesis = predictor.predict(example.src)
            meteor_score = self.example_score_meteor(' '.join(reference), ' '.join(hypothesis))
            results_prelim.append({
                'question': '"' + str(src) + '"',
                'reference': reference,
                'hypothesis': hypothesis,
                'meteor_score': meteor_score
            })
        results_meteor = [max((v for v in results_prelim if v['question'] == x), key=lambda y:y['meteor_score']) for x in set(v['question'] for v in results_prelim)] 

        with open(path+'result_meteor_output.txt', 'w') as f:
            for elem in results_meteor:
                f.write("%s\n" % elem)
                self.results_meteor.append(elem)
                self.meteor_score += elem['meteor_score']
                self.meteor_instances += 1
        return self.meteor_score/self.meteor_instances
    
    def average_meteor_score(self):
        """Return meteor average score"""
        return self.meteor_score/self.instances

    def reset(self):
        """Reset object properties"""
        self.results = []
        self.results_meteor = []
        self.score = 0
        self.meteor_score = 0
        self.instances = 0
        self.meteor_instances = 0

### Predictor

In [23]:
class Predictor(object):
    """Predictor class"""
    def __init__(self, model, src_vocab, trg_vocab, device):
        self.model = model
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.device = device

    def _predict_step(self, tokens):
        self.model.eval()
        tokenized_sentence = [SOS_TOKEN] + [t.lower() for t in tokens] + [EOS_TOKEN]
        numericalized = [self.src_vocab.stoi[token] for token in tokenized_sentence]
        src_tensor = torch.LongTensor(numericalized).unsqueeze(0).to(self.device)

        with torch.no_grad():
            encoder_out = self.model.encoder(src_tensor)

        outputs = [self.trg_vocab.stoi[SOS_TOKEN]]

        # cnn positional embedding gives assertion error for tensor
        # of size > max_positions-1, we predict tokens for max_positions-2
        # to avoid the error
        for _ in range(self.model.decoder.max_positions-2):
            trg_tensor = torch.LongTensor(outputs).unsqueeze(0).to(self.device)

            with torch.no_grad():
                output = self.model.decoder(trg_tensor, encoder_out, src_tokens=src_tensor)

            prediction = output.argmax(2)[:, -1].item()

            if prediction == self.trg_vocab.stoi[EOS_TOKEN]:
                break

            outputs.append(prediction)

        translation = [self.trg_vocab.itos[i] for i in outputs]

        return translation[1:] # , attention

    def _predict_rnn_step(self, tokens):
        self.model.eval()
        with torch.no_grad():
            tokenized_sentence = [SOS_TOKEN] + [t.lower() for t in tokens] + [EOS_TOKEN]
            numericalized = [self.src_vocab.stoi[t] for t in tokenized_sentence]

            src_len = torch.LongTensor([len(numericalized)]).to(self.device)
            tensor = torch.LongTensor(numericalized).unsqueeze(1).to(self.device)

            translation_tensor_logits = self.model(tensor.t(), src_len, None)

            translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1)
            translation = [self.trg_vocab.itos[t] for t in translation_tensor]

        return translation[1:] # , attention

    def predict(self, tokens):
        """Perform prediction on given tokens"""
        return self._predict_rnn_step(tokens) if self.model.name == RNN_NAME else \
                self._predict_step(tokens)

### Predict and Evaluate

In [26]:
model = Chechpoint.load(model,path,'./rnn.pt') # chọn path và tên model cho phù hợp

valid_iterator, test_iterator = BucketIterator.splits(
                                    (valid_data, test_data),
                                    batch_size=12,
                                    sort_within_batch=True, #if args.model == RNN_NAME else False,
                                    sort_key=lambda x: len(x.src),
                                    device=DEVICE)

# evaluate model
valid_loss = trainer.evaluator.evaluate(model, valid_iterator)
test_loss = trainer.evaluator.evaluate(model, test_iterator)

# calculate blue score for valid and test data
predictor = Predictor(model, src_vocab, trg_vocab, DEVICE)

valid_scorer = BleuScorer()
test_scorer = BleuScorer()
train_scorer = BleuScorer()
#bleu score
train_scorer.data_score(train_data.examples, predictor,path)
valid_scorer.data_score(valid_data.examples, predictor,path)
test_scorer.data_score(test_data.examples, predictor,path)

In [27]:
print(f'| Train Data Average BLEU score {train_scorer.average_score()*100} |')
print(f'| Train Data Average METEOR score {train_scorer.average_meteor_score()*100} |\n')
print(f'| Val. Loss: {valid_loss:.3f} | Test PPL: {math.exp(valid_loss):7.3f} |')
print(f'| Val. Data Average BLEU score {valid_scorer.average_score()*100} |')
print(f'| Val. Data Average METEOR score {valid_scorer.average_meteor_score()*100} |\n')
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')
print(f'| Test Data Average BLEU score {test_scorer.average_score()*100} |')
print(f'| Test Data Average METEOR score {test_scorer.average_meteor_score()*100} |')

In [28]:
for ex in test_data.examples[:50]:
    src_tmp = [t.lower() for t in ex.src]

    reference_tmp = [t.lower() for t in ex.trg]

    hypothesis_tmp = predictor.predict(ex.src)
    print(" ".join(src_tmp))
    print(" ".join(hypothesis_tmp))
    print(" ".join(reference_tmp),'\n')
    print('______________________________________________________________________________________________________________________________\n')